# 準備資料

In [2]:
import pandas as pd
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import os 


In [2]:
file_locate = "/tmp/pycharm_project_159/" #遠端環境
dataset_locate = file_locate+"/dataset/TA_model_FineTune_dataset/" 
images = os.listdir(file_locate + "/dataset/TA_model_FineTune_dataset/images")
Inputs = pd.read_csv(file_locate + "/dataset/TA_model_FineTune_dataset/question_Input.csv")

init_prompt = open(file_locate + "/dataset/init_prompt.txt").read()
rule = pd.read_excel(file_locate + "/dataset/回饋規則.xlsx")

#combine images and Inputs
def combine_images_and_inputs(images, Inputs):
    combined_data = []
    for i in range(len(images)):

        image_path = os.path.join(file_locate + "/dataset/TA_model_FineTune_dataset/images/", images[i])
        image = Image.open(image_path).convert("RGB")
        input_text = Inputs.iloc[i]["Input"]
        response = Inputs.iloc[i]["GroundTruth"]
        combined_data.append({"image": image, "input": input_text,"response":response} )

    return combined_data
dataset = combine_images_and_inputs(images, Inputs)

In [3]:
dataset

[{'image': <PIL.Image.Image image mode=RGB size=3040x2160>,
  'input': 'Pull Hook 左拉左曲球\n球速:66.43、發射角度:7.198、發射方向:-6.172、飛行距離:179、ClubAngleFace:-15、ClubAnglePath:6.901\nfront:{A: [0.0, 0.0, -0.0, 0.0, -0.0, -0.0], F: [-0.01, -0.008, 0.0, 0.048, -0.018, -0.011], I: [-0.019, -0.04, 0.184, -0.063, -0.041, -0.023], T: [0.002, 0.0, 0.0, -0.001, -0.001, -0.001]}\nside:{A: [0.002, -0.001, 0.001, -0.0, -0.002, 0.0], F: [0.001, 0.004, 0.0, -0.001, -0.003, 0.0], I: [0.217, -0.065, -0.013, -0.068, -0.054, -0.013], T: [-0.049, -0.043, 0.253, -0.083, -0.055, -0.013]}',
  'response': '        "球路": "Pull Hook 左拉左曲球",\n        "原因": "下桿角度過於陡峭，手腕過度彎曲，擊球時桿面過度關閉",\n        "建議": "在下桿（P5-P6）時，試著讓右肩與右手肘保持外旋，避免過度手腕翻轉，並確保手腕在P5.5時維持適當的屈曲角度，這有助於讓球桿的路徑更穩定。此外，擊球時（P7），確保左手腕恢復至較中立的狀態，以防止桿面過度關閉，造成左拉左曲球。"  '},
 {'image': <PIL.Image.Image image mode=RGB size=3040x2160>,
  'input': 'Fade 小右曲球\n球速:70.33、發射角度:9.56、發射方向:1.486、飛行距離:123.7、ClubAngleFace:2.66、ClubAnglePath:0.192\nfront:{A: [-0.0, 0.0, 0.0, -0.0, -0.0, -0.0]

In [4]:


# System message for the assistant
system_message = "You are an expert product description writer for Amazon."

# User prompt that combines the user query and the schema
user_prompt = """Create a Short Product description based on the provided <PRODUCT> and <CATEGORY> and image.
Only return description. The description should be SEO optimized and for a better mobile search experience.

<PRODUCT>
{product}
</PRODUCT>

<CATEGORY>
{category}
</CATEGORY>
"""

# Convert dataset to OAI messages
def format_data(sample):
    return {
        "messages": [
            {
                "role": "system",
                "content": [{"type": "text", "text": init_prompt}],
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": sample["input"],
                    },
                    {
                        "type": "image",
                        "image": sample["image"],
                    },
                ],
            },
            {
                "role": "assistant",
                "content": [{"type": "text", "text": sample["response"]}],
            },
        ],
    }

def process_vision_info(messages: list[dict]) -> list[Image.Image]:
    image_inputs = []
    # Iterate through each conversation
    for msg in messages:
        # Get content (ensure it's a list)
        content = msg.get("content", [])
        if not isinstance(content, list):
            content = [content]

        # Check each content element for images
        for element in content:
            if isinstance(element, dict) and (
                "image" in element or element.get("type") == "image"
            ):
                # Get the image and convert to RGB
                if "image" in element:
                    image = element["image"]
                else:
                    image = element
                image_inputs.append(image.convert("RGB"))
    return image_inputs



In [1]:

# Convert dataset to OAI messages
# need to use list comprehension to keep Pil.Image type, .mape convert image to bytes
dataset = [format_data(sample) for sample in dataset]

print(dataset[0]["messages"][0])

NameError: name 'dataset' is not defined

In [6]:
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig

/home/rikurzt/.virtualenvs/TESTSSH/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Hugging Face model id
model_id = "google/gemma-3-4b-pt" 

# Check if GPU benefits from bfloat16
if torch.cuda.get_device_capability()[0] < 8:
    raise ValueError("GPU does not support bfloat16, please use a GPU that supports bfloat16.")

# Define model init arguments
model_kwargs = dict(
    attn_implementation="eager", # Use "flash_attention_2" when running on Ampere or newer GPU
    torch_dtype=torch.bfloat16, # What torch dtype to use, defaults to auto
    device_map="auto", # Let torch decide how to load the model
)

# BitsAndBytesConfig int-4 config
model_kwargs["quantization_config"] = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=model_kwargs["torch_dtype"],
    bnb_4bit_quant_storage=model_kwargs["torch_dtype"],
)

# Load model and tokenizer
model = AutoModelForImageTextToText.from_pretrained(model_id, **model_kwargs)
processor = AutoProcessor.from_pretrained("google/gemma-3-4b-it",use_fast=True)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.67s/it]


In [8]:
from trl import SFTConfig

args = SFTConfig(
    output_dir="gemma-product-description",     # directory to save and repository id
    num_train_epochs=1,                         # number of training epochs
    per_device_train_batch_size=1,              # batch size per device during training
    gradient_accumulation_steps=4,              # number of steps before performing a backward/update pass
    gradient_checkpointing=True,                # use gradient checkpointing to save memory
    optim="adamw_torch_fused",                  # use fused adamw optimizer
    logging_steps=1,                            # log every 5 steps
    save_strategy="epoch",                      # save checkpoint every epoch
    learning_rate=2e-4,                         # learning rate, based on QLoRA paper
    bf16=True,                                  # use bfloat16 precision
    max_grad_norm=0.3,                          # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                          # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",               # use constant learning rate scheduler
    push_to_hub=False,                           # push model to hub
    report_to="tensorboard",                    # report metrics to tensorboard
    gradient_checkpointing_kwargs={
        "use_reentrant": False
    },  # use reentrant checkpointing
    dataset_text_field="",                      # need a dummy field for collator
    dataset_kwargs={"skip_prepare_dataset": True},  # important for collator
)
args.remove_unused_columns = False # important for collator

# Create a data collator to encode text and image pairs
def collate_fn(examples):
    texts = []
    images = []
    for example in examples:
        
        image_inputs = process_vision_info(example["messages"])
        text = processor.apply_chat_template(
            example["messages"], add_generation_prompt=False, tokenize=False
        )
        texts.append(text.strip())
        images.append(image_inputs)

    # Tokenize the texts and process the images
    batch = processor(text=texts, images=images, return_tensors="pt", padding=True)

    # The labels are the input_ids, and we mask the padding tokens and image tokens in the loss computation
    labels = batch["input_ids"].clone()

    # Mask image tokens
    image_token_id = [
        processor.tokenizer.convert_tokens_to_ids(
            processor.tokenizer.special_tokens_map["boi_token"]
        )
    ]
    # Mask tokens for not being used in the loss computation
    labels[labels == processor.tokenizer.pad_token_id] = -100
    labels[labels == image_token_id] = -100
    labels[labels == 262144] = -100

    batch["labels"] = labels
    return batch


In [9]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=16,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM",
    modules_to_save=[
        "lm_head",
        "embed_tokens",
    ],
)

In [10]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    peft_config=peft_config,
    processing_class=processor,
    data_collator=collate_fn,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [11]:
# Start training, the model will be automatically saved to the Hub and the output directory
trainer.train()

# Save the final model again to the Hugging Face Hub
trainer.save_model()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
1,9.615300


In [12]:
del model
del trainer
torch.cuda.empty_cache()

In [13]:
import torch

# Load Model with PEFT adapter
model = AutoModelForImageTextToText.from_pretrained(
  args.output_dir,
  device_map="cuda:0",
  torch_dtype=torch.bfloat16,
  attn_implementation="eager",
)
processor = AutoProcessor.from_pretrained(args.output_dir)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.02it/s]


In [14]:
from transformers import GenerationConfig
generation_config = GenerationConfig.from_pretrained("google/gemma-3-4b-pt")
generation_config.cache_implementation = "dynamic"

In [38]:
def generate_description(model, processor):
    # Convert sample into messages and then apply the chat template
    img = Image.open(dataset_locate+"/images/combined_186382.jpg").convert("RGB")
    messages = [
            {"role": "system","content": [{"type": "text", "text": init_prompt}],},
            {"role": "user",
              "content": [
                    {"type": "text", "text": dataset[0]['messages'][1]['content'][0]['text']},
                    {"type": "image", "image": img},
              ],
            },
    ]
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    # Process the image and text
    image_inputs = process_vision_info(messages)
    # Tokenize the text and process the images
    inputs = processor(
        text=[text],
        images=image_inputs,
        padding=True,
        return_tensors="pt",
    )
    # Move the inputs to the device
    inputs = inputs.to(model.device)
    
    # Generate the output
    stop_token_ids = [processor.tokenizer.eos_token_id, processor.tokenizer.convert_tokens_to_ids("<end_of_turn>")]
    generated_ids = model.generate(**inputs, max_new_tokens=1024, top_p=1.0, do_sample=True, temperature=0.3, eos_token_id=stop_token_ids, disable_compile=True)
    # Trim the generation and decode the output to text
    generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    return output_text[0]

# generate the description
description = generate_description(model, processor)
print(description)

你是一位充滿教學熱忱的高爾夫球教練，我會給你以下資訊:
1.某位學員在P2上桿(A)、P3上桿(A)、P4 頂點/轉換(F)、P5~6 下桿(I)、P7 擊球(T)的高爾夫擊球動作片段圖集，總共十張，上半部五張是學員正面的照片，下半部五張是學員側面的圖片，每張圖片紅色點與綠色的線代表學員的骨架，白色輔助線代表球員所在位置 ，粉色的輔助線代表球員的集求姿態不該超越的位置，橙色的輔助線代表揮桿夾角
2.學員的擊球數據，包含球速、發射角度、發射方向(球相對於目標線開始的初始方向。正發射方向表示球從目標右側開始，負發射方向表示球從目標左側開始)、飛行距離、ClubAngleFace(高爾夫球手將此稱為具有“開放”或“封閉”桿面。技術定義：在高爾夫球最大壓縮時，球桿面和高爾夫球接觸中心點的水平球桿面方向)、ClubAnglePath(高爾夫擊球的預期曲率(旋轉軸)的關鍵因素。假設中心接觸，球應該朝向面角彎曲並遠離球桿路徑(+5、0、-5))
3.與教練的正面和側面姿勢在A、F、I、T四個階段各個關節的差異，格式為front:{{A:[肩膀,手肘,手腕,髖部,膝蓋,腳踝],F:[......}}side:{{A:[肩膀,手肘,手腕,髖部,膝蓋,腳踝],F:[......}}
4.回饋規則
5.Clubface = clubAnglePath - clubAngleFace，大於3代表在原本的球路上還會偏左，小於3代表在原本的球路上還會遍右，例如左飛球碰到Clubface 大於3就會變成左拉左曲球
6.正確輸出建議模板:"......在我們全面的揮杆分析中，您的揮杆動作與教練大致相同，請繼續保持......"
7.失誤輸出建議模板:"......在我們全面的揮杆分析中，您的揮杆動作與教練最相似的地方在於<階段>，請繼續保持，然而您的揮杆有幾個地方可以加強。在您的<階段>、考慮調整您的<關節>，試著<調整方法>，這將有助於......"



你有以下幾項任務:
1.觀察學員正面與側面圖片的骨架以及參考線，找到姿勢不正確的圖片
2.判斷資訊3中與教練的姿勢差異是否有特別突出的地方，如果差異都不大，則判斷是那種正確球路，如果差異有特別突出的地方，則判斷是那種失誤球路
3.每次分析都需要根據任務2從回饋規則內尋找最相似的原因，結合Clubface以及圖片判斷屬於回饋規則內的哪一種球路類

In [41]:
def generate_description(model, processor):
    # Convert sample into messages and then apply the chat template
    img = Image.open(dataset_locate+"/images/combined_186382.jpg").convert("RGB")
    messages = [
            {"role": "system","content": [{"type": "text", "text": ""}],},
            {"role": "user",
              "content": [
                    {"type": "text", "text": "你看到什麼"},
                    {"type": "image", "image": img},
              ],
            },
    ]
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    # Process the image and text
    image_inputs = process_vision_info(messages)
    # Tokenize the text and process the images
    inputs = processor(
        text=[text],
        images=image_inputs,
        padding=True,
        return_tensors="pt",
    )
    # Move the inputs to the device
    inputs = inputs.to(model.device)
    
    # Generate the output
    stop_token_ids = [processor.tokenizer.eos_token_id, processor.tokenizer.convert_tokens_to_ids("<end_of_turn>")]
    generated_ids = model.generate(**inputs, max_new_tokens=1024, top_p=1.0, do_sample=True, temperature=0.3, eos_token_id=stop_token_ids, disable_compile=True)
    # Trim the generation and decode the output to text
    generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    return output_text[0]

# generate the description
description = generate_description(model, processor)
print(description)

枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫木
枫